## Clean your deployed model endpoint content
-----

#### In this notebook, we will parse through the existing endpoint.json to delete all of the existing endpoints once you are done with running your respective benchmarking tests.

***If you are with running all of the tests, and want to delete the existing endpoints, run this notebook.***

In [11]:
## Import all necessary libraries
import json
import boto3
import logging
from globals import *
from utils import load_config

In [12]:
## Set your logger to display all of the endpoints being cleaned
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Load the config file to extract our endpoint.json file and its respective file path
config = load_config(CONFIG_FILE)

In [13]:
## Parse through the existing endpoint.json file
## read the list of deployed endpoints that are active
endpoint_info_list = json.loads(Path(ENDPOINT_LIST_FPATH).read_text())
logger.info(json.dumps(endpoint_info_list, indent=2))

[2024-01-19 13:57:20,047] p1658 {885729754.py:4} INFO - [
  {
    "experiment_name": "llama2-13b-p4d.24xlarge-tgi-inference-2.0.1-tgi0.9.3-gpu-py39-cu118",
    "endpoint": {
      "EndpointName": "huggingface-pytorch-tgi-inference-2024-01-19-06-20-11-080",
      "EndpointArn": "arn:aws:sagemaker:us-east-1:015469603702:endpoint/huggingface-pytorch-tgi-inference-2024-01-19-06-20-11-080",
      "EndpointConfigName": "huggingface-pytorch-tgi-inference-2024-01-19-06-20-11-080",
      "ProductionVariants": [
        {
          "VariantName": "AllTraffic",
          "DeployedImages": [
            {
              "SpecifiedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi0.9.3-gpu-py39-cu118-ubuntu20.04",
              "ResolvedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference@sha256:947399ae8b3fa131fc6d2da99f56c9c41195c7ce7cbd890e1e6c0dc328d238cd",
              "ResolutionTime": "2024-01-19 06:20:12.62500

In [14]:
## initialize a sagemaker client
sm_client = boto3.client("sagemaker")
# Iterate over the endpoint_info_list and mark the items for deletion
for item in endpoint_info_list:   
    
    ## Extract the endpoint name from the deployed model configuration
    ep_name = item['endpoint']["EndpointName"]
    try:
        ## Describe the model endpoint 
        logger.info(f"Going to describing the endpoint -> {ep_name}")
        resp = sm_client.describe_endpoint(EndpointName=ep_name)
        
        ## If the given model endpoint is in service, delete it 
        if resp['EndpointStatus'] == 'InService':
            logger.info(f"going to delete {ep_name}")
            ## deleting the model endpoint
            sm_client.delete_endpoint(EndpointName=ep_name)
            logger.info(f"deleted {ep_name}")
    except Exception as e:
        logger.error(f"error deleting endpoint={ep_name}, exception={e}")

[2024-01-19 13:57:20,134] p1658 {707195403.py:10} INFO - Going to describing the endpoint -> huggingface-pytorch-tgi-inference-2024-01-19-06-20-11-080
[2024-01-19 13:57:20,305] p1658 {707195403.py:20} ERROR - error deleting endpoint=huggingface-pytorch-tgi-inference-2024-01-19-06-20-11-080, exception=An error occurred (ValidationException) when calling the DescribeEndpoint operation: Could not find endpoint "huggingface-pytorch-tgi-inference-2024-01-19-06-20-11-080".
[2024-01-19 13:57:20,306] p1658 {707195403.py:10} INFO - Going to describing the endpoint -> llama-2-13b-inf2-24xlarge-1705642890
[2024-01-19 13:57:20,447] p1658 {707195403.py:20} ERROR - error deleting endpoint=llama-2-13b-inf2-24xlarge-1705642890, exception=An error occurred (ValidationException) when calling the DescribeEndpoint operation: Could not find endpoint "llama-2-13b-inf2-24xlarge-1705642890".
[2024-01-19 13:57:20,448] p1658 {707195403.py:10} INFO - Going to describing the endpoint -> llama-2-13b-inf2-48xlarge-